In [1]:
# visualization
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
fe = fm.FontEntry(
    fname=r'/usr/share/fonts/truetype/nanum/NanumGothic.ttf', # ttf 파일이 저장되어 있는 경로
    name='NanumBarunGothic')                        # 이 폰트의 원하는 이름 설정
fm.fontManager.ttflist.insert(0, fe)              # Matplotlib에 폰트 추가
plt.rcParams.update({'font.size': 10, 'font.family': 'NanumBarunGothic'}) # 폰트 설정
plt.rc('font', family='NanumBarunGothic')
import seaborn as sns
from math import radians, sin, cos, sqrt, atan2

# utils
import pandas as pd
import numpy as np
from tqdm import tqdm
import pickle
import warnings;warnings.filterwarnings('ignore')

# Model
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics
from sklearn.neighbors import BallTree
from lightgbm import LGBMRegressor

import eli5
from eli5.sklearn import PermutationImportance

import re
import os

##### Data Load

In [2]:
with open("./Data/01_eda_end/preprocessed_concat.pkl", "rb") as f:
    df = pickle.load(f)

print("Shape:", df.shape)
print("Columns 예시:", list(df.columns)[:15])

# 외부데이터 불러오기
bus = pd.read_csv('./Data/bus_feature.csv')
subway = pd.read_csv('./Data/subway_feature.csv')
elementary = pd.read_csv('./Data/elementary_school_feature.csv')
middle = pd.read_csv('./Data/middle_school_feature.csv')
high = pd.read_csv('./Data/high_school_feature.csv')
interest = pd.read_csv('./Data/base_interest_rate.csv')
mortgage = pd.read_csv('./Data/mortgage_rate.csv')

Shape: (1128094, 52)
Columns 예시: ['시군구', '번지', '본번', '부번', '아파트명', '전용면적', '계약년월', '계약일', '층', '건축년도', '도로명', '해제사유발생일', '등기신청일자', '거래유형', '중개사소재지']


In [3]:
# train / test 분리
train = df.query("is_test == 0").copy()
test  = df.query("is_test == 1").copy()

print("train shape:", train.shape)
print("test  shape:", test.shape)

train shape: (1118822, 52)
test  shape: (9272, 52)


In [4]:
# 시/구/동 컬럼 다시 생성
addr_split = df["시군구"].astype(str).str.split()

df["시"] = addr_split.str[0]
df["구"] = addr_split.str[1]
df["동"] = addr_split.str[2]

print("▶ 시/구/동 생성 예시")
print(df[["시군구", "시", "구", "동"]].head())
print("\n구 unique 개수:", df["구"].nunique())
print("동 unique 개수:", df["동"].nunique())

# 혹시 모르니까 다시 한번 분리
train = df.query("is_test == 0").copy()
test = df.query("is_test == 1").copy()

▶ 시/구/동 생성 예시
             시군구      시    구    동
0  서울특별시 강남구 개포동  서울특별시  강남구  개포동
1  서울특별시 강남구 개포동  서울특별시  강남구  개포동
2  서울특별시 강남구 개포동  서울특별시  강남구  개포동
3  서울특별시 강남구 개포동  서울특별시  강남구  개포동
4  서울특별시 강남구 개포동  서울특별시  강남구  개포동

구 unique 개수: 25
동 unique 개수: 337


## 파생변수 생성

##### Part1. 입지 기반 파생변수

- 1-1. 구 단위 가격

In [5]:
# 1-1.구 단위 가격 변수 생성
df_train = df[df['is_test'] == 0].copy()

# 구별 평균 / 중위값 / 거래량
gu_stats = df_train.groupby('구')['target'].agg(
    구_평균가격='mean',
    구_중위가격='median',
    구_거래량='count'
).reset_index()

# 원본 df에 merge
df = df.merge(gu_stats, on='구', how='left')

print("파생변수 생성 후 df shape:", df.shape)
df[['구','구_평균가격','구_중위가격','구_거래량']].head()

파생변수 생성 후 df shape: (1128094, 58)


,구,구_평균가격,구_중위가격,구_거래량
0,강남구,114847.461503,93000.0,69083
1,강남구,114847.461503,93000.0,69083
2,강남구,114847.461503,93000.0,69083
3,강남구,114847.461503,93000.0,69083
4,강남구,114847.461503,93000.0,69083


- 1-2. 동 단위 가격

In [6]:
# 1-2.동 단위 가격 변수 생성 (평균/거래량)
dong_stats = (
    train.groupby("동")["target"]
         .agg(동_평균가격="mean", 동_거래량="count")
         .reset_index()
)

print("동 단위 통계:")
display(dong_stats.head())

# df 전체에 merge
df = df.merge(dong_stats, on="동", how="left")

print("파생변수 생성 후 df shape:", df.shape)
df[["동", "동_평균가격", "동_거래량"]].head()

동 단위 통계:


,동,동_평균가격,동_거래량
0,가락동,66266.579598,12940
1,가리봉동,28246.831683,101
2,가산동,29375.567834,2027
3,가양동,39457.146194,9590
4,갈월동,60392.857143,14


파생변수 생성 후 df shape: (1128094, 60)


,동,동_평균가격,동_거래량
0,개포동,92694.455557,12218
1,개포동,92694.455557,12218
2,개포동,92694.455557,12218
3,개포동,92694.455557,12218
4,개포동,92694.455557,12218


- 1-3. 강남여부

In [7]:
# 1-3.강남 여부 생성
gangnam_list = ["강남구", "서초구", "송파구", "강동구"]

df["강남여부"] = df["구"].apply(lambda x: 1 if x in gangnam_list else 0)

print(df["강남여부"].value_counts())
df[["구", "강남여부"]].head()

강남여부
0    868016
1    260078
Name: count, dtype: int64


,구,강남여부
0,강남구,1
1,강남구,1
2,강남구,1
3,강남구,1
4,강남구,1


1-4. 3대 업무지구 (CBD, GBD, YBD)

In [8]:
# 1-4.3대업무지구 CBD, GBD, YBD 여부 생성
CBD = ["종로구", "중구"]
GBD = ["강남구", "서초구"]
YBD = ["영등포구"]  # 여의도동은 동 단위에서 체크 가능하지만, 기본은 영등포구로 처리

df["CBD_여부"] = df["구"].apply(lambda x: 1 if x in CBD else 0)
df["GBD_여부"] = df["구"].apply(lambda x: 1 if x in GBD else 0)

# YBD는 location이 구 단위로 포함되므로 동일하게 적용
df["YBD_여부"] = df["구"].apply(lambda x: 1 if x in YBD else 0)

print("CBD/GBD/YBD 여부 요약:")
print(df[["구", "CBD_여부", "GBD_여부", "YBD_여부"]].head())

CBD/GBD/YBD 여부 요약:
     구  CBD_여부  GBD_여부  YBD_여부
0  강남구       0       1       0
1  강남구       0       1       0
2  강남구       0       1       0
3  강남구       0       1       0
4  강남구       0       1       0


- 1.5. 지하철역 관련 파생변수

In [9]:
# 1-5. 지하철역 관련 파생여부 생성 전 해당 컬럼 데이터 프레임 확인
print("Subway columns:", subway.columns.tolist())
subway.head()

Subway columns: ['역사_ID', '역사명', '호선', '위도', '경도']


,역사_ID,역사명,호선,위도,경도
0,9996,미사,5호선,37.560927,127.193877
1,9995,강일,5호선,37.557490,127.175930
2,4929,김포공항,김포골드라인,37.562360,126.801868
3,4928,고촌,김포골드라인,37.601243,126.770345
4,4927,풍무,김포골드라인,37.612488,126.732387


In [10]:
# BallTree 용 좌표 전처리
EARTH_RADIUS_KM = 6371.0

# 1. 지하철 좌표 (위도, 경도) → 라디안
subway_coords_rad = np.radians(subway[["위도", "경도"]].values)

print("subway_coords_rad shape:", subway_coords_rad.shape)

# 2. 아파트 좌표 중 결측 아닌 것만 사용
mask_has_coord = df["좌표X"].notnull() & df["좌표Y"].notnull()
apt_coords = df.loc[mask_has_coord, ["좌표Y", "좌표X"]].values  # (위도, 경도) 순서

# 3. 라디안 변환
apt_coords_rad = np.radians(apt_coords)

print("아파트 좌표 사용 개수:", apt_coords_rad.shape[0])

subway_coords_rad shape: (768, 2)
아파트 좌표 사용 개수: 251862


In [11]:
# 최근접 지하철 거리 계산
# 1. BallTree 구축 (Haversine 거리)
subway_tree = BallTree(subway_coords_rad, metric="haversine")

# 2. 각 아파트에서 가장 가까운 지하철까지의 거리 (라디안)
#    dist_rad.shape = (N, 1), ind.shape = (N, 1)
dist_rad, ind = subway_tree.query(apt_coords_rad, k=1)

# 3. km 단위로 변환
dist_km = dist_rad.flatten() * EARTH_RADIUS_KM

print("예시 거리 (km) 10개:", dist_km[:10])

# 4. df에 반영
df["subway_min_dist_km"] = np.nan
df.loc[mask_has_coord, "subway_min_dist_km"] = dist_km.astype("float32")

df[["subway_min_dist_km"]].describe()

예시 거리 (km) 10개: [1.1297748 1.1297748 1.1297748 1.1297748 1.1297748 1.1297748 1.1297748
 1.1297748 1.1297748 1.1297748]


,subway_min_dist_km
count,251862.000000
mean,0.559903
std,0.323998
min,0.036173
25%,0.332314
50%,0.487873
75%,0.714205
max,2.510675


In [12]:
# 역세권 여부(500m) 플래그 만들기
df["subway_is_500m"] = 0
df.loc[mask_has_coord & (df["subway_min_dist_km"] <= 0.5), "subway_is_500m"] = 1

print(df["subway_is_500m"].value_counts(dropna=False))

subway_is_500m
0    997518
1    130576
Name: count, dtype: int64


In [13]:
# 반경 내 지하철역 개수 (기준 500m / 1km)
# 반경 (km) → 라디안으로 변환
radius_500m_rad = 0.5 / EARTH_RADIUS_KM
radius_1km_rad  = 1.0 / EARTH_RADIUS_KM

# 1. 500m 반경 내 지하철역 개수
indices_500m = subway_tree.query_radius(apt_coords_rad, r=radius_500m_rad)
cnt_500m = np.array([len(idx) for idx in indices_500m], dtype="int16")

# 2. 1km 반경 내 지하철역 개수
indices_1km = subway_tree.query_radius(apt_coords_rad, r=radius_1km_rad)
cnt_1km = np.array([len(idx) for idx in indices_1km], dtype="int16")

# 3. df에 반영
df["subway_cnt_500m"] = 0
df["subway_cnt_1km"] = 0

df.loc[mask_has_coord, "subway_cnt_500m"] = cnt_500m
df.loc[mask_has_coord, "subway_cnt_1km"] = cnt_1km

df[["subway_cnt_500m", "subway_cnt_1km"]].describe()

,subway_cnt_500m,subway_cnt_1km
count,1.128094e+06,1.128094e+06
mean,1.510202e-01,6.312116e-01
std,4.693136e-01,1.504500e+00
min,0.000000e+00,0.000000e+00
25%,0.000000e+00,0.000000e+00
50%,0.000000e+00,0.000000e+00
75%,0.000000e+00,0.000000e+00
max,6.000000e+00,1.200000e+01


- 1.6. 버스 관련 파생변수

In [14]:
# 버스 데이터 컬럼 목록 확인
print("버스 데이터 shape:", bus.shape)
print("\n버스 데이터 컬럼 목록:")
print(bus.columns.tolist())

print("\n버스 데이터 상위 5개:")
bus.head()

버스 데이터 shape: (12584, 6)

버스 데이터 컬럼 목록:
['노드 ID', '정류소번호', '정류소명', 'X좌표', 'Y좌표', '정류소 타입']

버스 데이터 상위 5개:


,노드 ID,정류소번호,정류소명,X좌표,Y좌표,정류소 타입
0,100000001,1001,종로2가사거리,126.987752,37.569808,중앙차로
1,100000002,1002,창경궁.서울대학교병원,126.996566,37.579183,중앙차로
2,100000003,1003,명륜3가.성대입구,126.998251,37.582581,중앙차로
3,100000004,1004,종로2가.삼일교,126.987613,37.568579,중앙차로
4,100000005,1005,혜화동로터리.여운형활동터,127.001744,37.586243,중앙차로


In [15]:
# 유효한 버스 좌표 필터링
bus_valid = bus.dropna(subset=['X좌표', 'Y좌표']).copy()

print("유효 버스 정류장 수:", len(bus_valid))
print(bus_valid.head())

유효 버스 정류장 수: 12584
       노드 ID  정류소번호           정류소명         X좌표        Y좌표 정류소 타입
0  100000001   1001        종로2가사거리  126.987752  37.569808   중앙차로
1  100000002   1002    창경궁.서울대학교병원  126.996566  37.579183   중앙차로
2  100000003   1003      명륜3가.성대입구  126.998251  37.582581   중앙차로
3  100000004   1004       종로2가.삼일교  126.987613  37.568579   중앙차로
4  100000005   1005  혜화동로터리.여운형활동터  127.001744  37.586243   중앙차로


In [16]:
# 지하철역 처럼 HaverSine / BallTree 를 이용하기 위해서 라디안 변환 + 넘파이 배열 준비
# (위도, 경도) 순서로 정렬
bus_coords_rad = np.radians(bus_valid[['Y좌표', 'X좌표']].to_numpy())

print("버스 좌표 shape:", bus_coords_rad.shape)

버스 좌표 shape: (12584, 2)


In [17]:
# 아파트 좌표 셋팅
mask_has_coord = df['좌표Y'].notnull() & df['좌표X'].notnull()

apt_coords_rad = np.radians(
    df.loc[mask_has_coord, ['좌표Y', '좌표X']].to_numpy()
)

print("아파트 좌표 개수:", len(apt_coords_rad))

아파트 좌표 개수: 251862


In [18]:
# BallTree 생성
R = 6371  # 지구 반경 km
bus_tree = BallTree(bus_coords_rad, metric='haversine')

# 가장 가까운 버스 정류장 거리 계산
dist_rad, _ = bus_tree.query(apt_coords_rad, k=1)
dist_km = dist_rad.flatten() * R

df.loc[mask_has_coord, 'bus_min_dist_km'] = dist_km

# 결측치(좌표 없는 행)는 중앙값으로 채움
df['bus_min_dist_km'] = df['bus_min_dist_km'].fillna(df['bus_min_dist_km'].median())

In [19]:
# 반경 내 버스 정류장 개수 계산 (300m / 500m)
radius_300 = 0.3 / R
radius_500 = 0.5 / R

# 300m count
idx_300 = bus_tree.query_radius(apt_coords_rad, r=radius_300)
df.loc[mask_has_coord, 'bus_cnt_300m'] = [len(i) for i in idx_300]

# 500m count
idx_500 = bus_tree.query_radius(apt_coords_rad, r=radius_500)
df.loc[mask_has_coord, 'bus_cnt_500m'] = [len(i) for i in idx_500]

# 결측은 0으로
df['bus_cnt_300m'] = df['bus_cnt_300m'].fillna(0).astype(int)
df['bus_cnt_500m'] = df['bus_cnt_500m'].fillna(0).astype(int)

In [20]:
# 버스 접근성 flag
df['bus_is_300m'] = (df['bus_min_dist_km'] <= 0.3).astype(int)
df['bus_is_500m'] = (df['bus_min_dist_km'] <= 0.5).astype(int)

In [21]:
# 결과 검증
print(df[['bus_min_dist_km', 'bus_cnt_300m', 'bus_cnt_500m']].describe())

print("\n300m 내 정류장 개수 분포:")
print(df['bus_cnt_300m'].value_counts().head())

print("\n500m 내 정류장 개수 분포:")
print(df['bus_cnt_500m'].value_counts().head())

       bus_min_dist_km  bus_cnt_300m  bus_cnt_500m
count     1.128094e+06  1.128094e+06  1.128094e+06
mean      1.184303e-01  1.927236e+00  5.251618e+00
std       2.847471e-02  4.296405e+00  1.090393e+01
min       7.063893e-03  0.000000e+00  0.000000e+00
25%       1.166271e-01  0.000000e+00  0.000000e+00
50%       1.166271e-01  0.000000e+00  0.000000e+00
75%       1.166271e-01  0.000000e+00  0.000000e+00
max       4.712484e-01  3.200000e+01  7.500000e+01

300m 내 정류장 개수 분포:
bus_cnt_300m
0    879160
6     26457
7     26254
8     21587
4     18525
Name: count, dtype: int64

500m 내 정류장 개수 분포:
bus_cnt_500m
0     876232
22     17705
25     15438
28     12761
18     12262
Name: count, dtype: int64


- 1.7. 초,중,고등학교 관련 파생변수

In [22]:
# 기본 정보 출력
print("===== 초등학교 데이터 =====")
print("shape:", elementary.shape)
print("columns:", elementary.columns.tolist())
print(elementary.head(), "\n")

print("===== 중학교 데이터 =====")
print("shape:", middle.shape)
print("columns:", middle.columns.tolist())
print(middle.head(), "\n")

print("===== 고등학교 데이터 =====")
print("shape:", high.shape)
print("columns:", high.columns.tolist())
print(high.head(), "\n")

===== 초등학교 데이터 =====
shape: (614, 29)
columns: ['시도교육청', '교육지원청', '지역', '정보공시 학교코드', '학교명', '학교급코드', '설립구분', '학교특성', '분교여부', '설립유형', '주야구분', '개교기념일', '설립일', '법정동코드', '주소내역', '상세주소내역', '우편번호', '학교도로명 우편번호', '학교도로명 주소', '학교도로명 상세주소', '위도', '경도', '전화번호', '팩스번호', '홈페이지 주소', '남녀공학 구분', '폐교여부', '폐교일자', '휴교여부']
      시도교육청           교육지원청         지역   정보공시 학교코드              학교명  학교급코드  \
0       교육부  서울특별시강남서초교육지원청  서울특별시 서초구  S000003511    서울교육대학교부설초등학교      2   
1       교육부             교육부  서울특별시 종로구  S000003563  서울대학교사범대학부설초등학교      2   
2  서울특별시교육청  서울특별시강남서초교육지원청  서울특별시 강남구  S010000737         서울개원초등학교      2   
3  서울특별시교육청  서울특별시강남서초교육지원청  서울특별시 강남구  S010000738         서울개일초등학교      2   
4  서울특별시교육청  서울특별시강남서초교육지원청  서울특별시 강남구  S010000739         서울개포초등학교      2   

  설립구분  학교특성 분교여부 설립유형  ...             학교도로명 상세주소         위도          경도  \
0   국립   NaN    N   부설  ...         (서초동,서울교대초등학교)  37.490739  127.015424   
1   국립   NaN    N   부설  ...       (동숭동,서울사대부설초등학교)  37.577791  127.0028

In [23]:
# 시군구 → 구 컬럼이 이미 있다면 유지, 없으면 새로 생성
if '구' not in df.columns:
    if '시군구' in df.columns:
        addr_split = df['시군구'].astype(str).str.split()
        df['구'] = addr_split.str[1]  # 예: "서울특별시 강남구" → ['서울특별시','강남구']
        print("✅ df['구'] 컬럼 새로 생성 완료")
    else:
        raise ValueError("df에 '구' 컬럼도 없고 '시군구' 컬럼도 없어서 구 정보를 만들 수 없습니다.")

print("df['구'] 예시:", df['구'].head())


df['구'] 예시: 0    강남구
1    강남구
2    강남구
3    강남구
4    강남구
Name: 구, dtype: object


In [24]:
R = 6371.0  # 지구 반지름 (km)

# 초/중/고 데이터는 이전 셸에서 elem_df, mid_df, high_df 로 로드되어 있다고 가정
print("elem_df shape:", elementary.shape)
print("mid_df shape:", middle.shape)
print("high_df shape:", high.shape)

# 결측 좌표 제거 (혹시 모를 대비)
elem_valid = elementary.dropna(subset=['위도', '경도']).copy()
mid_valid  = middle.dropna(subset=['위도', '경도']).copy()
high_valid = high.dropna(subset=['위도', '경도']).copy()

print("유효 초등학교 수:", len(elem_valid))
print("유효 중학교 수:", len(mid_valid))
print("유효 고등학교 수:", len(high_valid))

elem_df shape: (614, 29)
mid_df shape: (392, 29)
high_df shape: (322, 29)
유효 초등학교 수: 613
유효 중학교 수: 391
유효 고등학교 수: 321


In [25]:
# 각 학교 레벨에 대해 (위도, 경도) → 라디안 변환
elem_coords_rad  = np.radians(elem_valid[['위도', '경도']].to_numpy())
mid_coords_rad   = np.radians(mid_valid[['위도', '경도']].to_numpy())
high_coords_rad  = np.radians(high_valid[['위도', '경도']].to_numpy())

print("elem_coords_rad shape:", elem_coords_rad.shape)
print("mid_coords_rad shape:", mid_coords_rad.shape)
print("high_coords_rad shape:", high_coords_rad.shape)

elem_coords_rad shape: (613, 2)
mid_coords_rad shape: (391, 2)
high_coords_rad shape: (321, 2)


In [26]:
# 아파트 좌표 라디안 변환
mask_has_coord = df['좌표Y'].notnull() & df['좌표X'].notnull()
apt_coords_rad = np.radians(
    df.loc[mask_has_coord, ['좌표Y', '좌표X']].to_numpy()
)

print("사용할 아파트 좌표 개수:", len(apt_coords_rad))

사용할 아파트 좌표 개수: 251862


In [27]:
# 초/중/고 각각 최근접 학교 거리 계산
# 초등학교 BallTree
elem_tree = BallTree(elem_coords_rad, metric='haversine')
dist_elem_rad, _ = elem_tree.query(apt_coords_rad, k=1)
dist_elem_km = dist_elem_rad.flatten() * R

# 중학교 BallTree
mid_tree = BallTree(mid_coords_rad, metric='haversine')
dist_mid_rad, _ = mid_tree.query(apt_coords_rad, k=1)
dist_mid_km = dist_mid_rad.flatten() * R

# 고등학교 BallTree
high_tree = BallTree(high_coords_rad, metric='haversine')
dist_high_rad, _ = high_tree.query(apt_coords_rad, k=1)
dist_high_km = dist_high_rad.flatten() * R

print("초등학교 거리 예시(10개):", dist_elem_km[:10])
print("중학교 거리 예시(10개):", dist_mid_km[:10])
print("고등학교 거리 예시(10개):", dist_high_km[:10])

초등학교 거리 예시(10개): [0.46146105 0.46146105 0.46146105 0.46146105 0.46146105 0.46146105
 0.46146105 0.46146105 0.46146105 0.46146105]
중학교 거리 예시(10개): [0.5900958 0.5900958 0.5900958 0.5900958 0.5900958 0.5900958 0.5900958
 0.5900958 0.5900958 0.5900958]
고등학교 거리 예시(10개): [0.50948569 0.50948569 0.50948569 0.50948569 0.50948569 0.50948569
 0.50948569 0.50948569 0.50948569 0.50948569]


In [28]:
# 기본값 NaN으로 두고, 좌표 있는 행에만 값을 채워넣기
df['elem_min_dist_km']  = np.nan
df['mid_min_dist_km']   = np.nan
df['high_min_dist_km']  = np.nan

df.loc[mask_has_coord, 'elem_min_dist_km']  = dist_elem_km.astype('float32')
df.loc[mask_has_coord, 'mid_min_dist_km']   = dist_mid_km.astype('float32')
df.loc[mask_has_coord, 'high_min_dist_km']  = dist_high_km.astype('float32')

df[['elem_min_dist_km', 'mid_min_dist_km', 'high_min_dist_km']].describe()

,elem_min_dist_km,mid_min_dist_km,high_min_dist_km
count,251862.000000,251862.000000,251862.000000
mean,0.330902,0.463148,0.640123
std,0.148502,0.243301,0.388066
min,0.043141,0.053590,0.079784
25%,0.211378,0.275021,0.364277
50%,0.312486,0.415582,0.571693
75%,0.441992,0.600737,0.818976
max,0.943855,1.538712,3.016146


In [29]:
# 가장 가까운 학교까지 거리 통합 변수 만들기
# 세 레벨 중 최소 거리
df['school_min_dist_km'] = df[
    ['elem_min_dist_km', 'mid_min_dist_km', 'high_min_dist_km']
].min(axis=1)

df['school_min_dist_km'].describe()

count    251862.000000
mean          0.280994
std           0.129957
min           0.043141
25%           0.180454
50%           0.253579
75%           0.366362
max           0.943855
Name: school_min_dist_km, dtype: float64

In [30]:
# 구 단위 학교 수 파생변수 제작
def extract_gu_from_region(s):
    # 예: "서울특별시 강남구" → ['서울특별시','강남구'] → '강남구'
    if isinstance(s, str):
        parts = s.split()
        if len(parts) >= 2:
            return parts[1]
    return np.nan

elem_valid['구']  = elem_valid['지역'].apply(extract_gu_from_region)
mid_valid['구']   = mid_valid['지역'].apply(extract_gu_from_region)
high_valid['구']  = high_valid['지역'].apply(extract_gu_from_region)

print(elem_valid['구'].value_counts().head())
print(mid_valid['구'].value_counts().head())
print(high_valid['구'].value_counts().head())

구
노원구    42
송파구    41
강서구    36
강남구    34
은평구    32
Name: count, dtype: int64
구
송파구    29
노원구    26
강남구    24
강서구    23
강동구    19
Name: count, dtype: int64
구
노원구    25
강서구    23
강남구    22
송파구    21
은평구    18
Name: count, dtype: int64


In [31]:
# merge
# 구별 초/중/고 학교 수 집계
elem_gu_cnt = elem_valid.groupby('구').size().reset_index(name='elem_school_cnt')
mid_gu_cnt  = mid_valid.groupby('구').size().reset_index(name='mid_school_cnt')
high_gu_cnt = high_valid.groupby('구').size().reset_index(name='high_school_cnt')

# 하나의 테이블로 합치기
gu_school_stats = elem_gu_cnt.merge(mid_gu_cnt, on='구', how='outer') \
                             .merge(high_gu_cnt, on='구', how='outer')

# NaN → 0
for col in ['elem_school_cnt', 'mid_school_cnt', 'high_school_cnt']:
    gu_school_stats[col] = gu_school_stats[col].fillna(0).astype(int)

print(gu_school_stats.head())

     구  elem_school_cnt  mid_school_cnt  high_school_cnt
0  강남구               34              24               22
1  강동구               29              19               14
2  강북구               14              13                7
3  강서구               36              23               23
4  관악구               22              16               17


In [32]:
# df의 '구' 기준으로 학교 수 정보 merge
df = df.merge(gu_school_stats, on='구', how='left')

# 혹시 구 정보가 없는 행은 0으로 처리
for col in ['elem_school_cnt', 'mid_school_cnt', 'high_school_cnt']:
    df[col] = df[col].fillna(0).astype(int)

df[['구', 'elem_school_cnt', 'mid_school_cnt', 'high_school_cnt']].head()

,구,elem_school_cnt,mid_school_cnt,high_school_cnt
0,강남구,34,24,22
1,강남구,34,24,22
2,강남구,34,24,22
3,강남구,34,24,22
4,강남구,34,24,22


In [33]:
print("학교까지 최소 거리 통계")
print(df['school_min_dist_km'].describe())

print("\n구별 학교 수 분포 (초/중/고)")
print(df[['elem_school_cnt', 'mid_school_cnt', 'high_school_cnt']].describe())

학교까지 최소 거리 통계
count    251862.000000
mean          0.280994
std           0.129957
min           0.043141
25%           0.180454
50%           0.253579
75%           0.366362
max           0.943855
Name: school_min_dist_km, dtype: float64

구별 학교 수 분포 (초/중/고)
       elem_school_cnt  mid_school_cnt  high_school_cnt
count     1.128094e+06    1.128094e+06     1.128094e+06
mean      2.806341e+01    1.791034e+01     1.473140e+01
std       8.141317e+00    5.651974e+00     6.024011e+00
min       1.200000e+01    8.000000e+00     6.000000e+00
25%       2.200000e+01    1.400000e+01     1.000000e+01
50%       2.600000e+01    1.600000e+01     1.400000e+01
75%       3.400000e+01    2.300000e+01     2.100000e+01
max       4.200000e+01    2.900000e+01     2.500000e+01


##### Part2. 건물특성 기반 파생변수

In [34]:
# 변수 생성 전 기본 분포 확인
print("건축년도 기본 통계")
print(df["건축년도"].describe())

print("\n층 기본 통계")
print(df["층"].describe())

print("\n건축년도 고유값 상위 20개:")
print(sorted(df["건축년도"].unique())[:20], "...")

print("\n층 고유값 상위 20개:")
print(sorted(df["층"].unique())[:20], "...")

건축년도 기본 통계
count    1.128094e+06
mean     1.998791e+03
std      9.358540e+00
min      1.961000e+03
25%      1.992000e+03
50%      2.000000e+03
75%      2.005000e+03
max      2.023000e+03
Name: 건축년도, dtype: float64

층 기본 통계
count    1.128094e+06
mean     8.881609e+00
std      5.989011e+00
min     -1.000000e+00
25%      4.000000e+00
50%      8.000000e+00
75%      1.200000e+01
max      6.900000e+01
Name: 층, dtype: float64

건축년도 고유값 상위 20개:
[1961, 1965, 1966, 1967, 1968, 1969, 1970, 1971, 1972, 1973, 1974, 1975, 1976, 1977, 1978, 1979, 1980, 1981, 1982, 1983] ...

층 고유값 상위 20개:
[-1, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19] ...


- 2-1. 건물 나이 및 노후도 구간 파생변수

In [35]:
# 1. 건물 나이 계산 (기준연도: 2023)
BASE_YEAR = 2023
df["building_age"] = BASE_YEAR - df["건축년도"]

print("building_age 기본 통계")
print(df["building_age"].describe())

# 2. 노후도 구간 파생변수 (신축 / 준노후 / 노후)
def categorize_age(age):
    if pd.isna(age):
        return "unknown"
    if age < 15:
        return "신축(<15년)"
    elif age < 30:
        return "준노후(15~30년)"
    else:
        return "노후(30년 이상)"

df["building_age_bucket"] = df["building_age"].apply(categorize_age)

print("\n노후도 구간 분포:")
print(df["building_age_bucket"].value_counts(dropna=False))

# 3. 이진 플래그 파생변수 (신축 여부, 재건축 가능 여부)
df["is_new"] = (df["building_age"] < 15).astype(int)
df["is_old_redev"] = (df["building_age"] >= 30).astype(int)

print("\n신축 여부(is_new) 분포:")
print(df["is_new"].value_counts())

print("\n재건축 가능 여부(is_old_redev) 분포:")
print(df["is_old_redev"].value_counts())


building_age 기본 통계
count    1.128094e+06
mean     2.420950e+01
std      9.358540e+00
min      0.000000e+00
25%      1.800000e+01
50%      2.300000e+01
75%      3.100000e+01
max      6.200000e+01
Name: building_age, dtype: float64

노후도 구간 분포:
building_age_bucket
준노후(15~30년)    660820
노후(30년 이상)     315221
신축(<15년)       152053
Name: count, dtype: int64

신축 여부(is_new) 분포:
is_new
0    976041
1    152053
Name: count, dtype: int64

재건축 가능 여부(is_old_redev) 분포:
is_old_redev
0    812873
1    315221
Name: count, dtype: int64


In [36]:
# 3. 건축년도 구간화 + 비선형 변환 (연도 기준 구간화)
year_bins = [1960, 1980, 1990, 2000, 2010, 2015, 2020, 2025]
year_labels = [
    "1980년 이전",
    "1980~1989년",
    "1990~1999년",
    "2000~2009년",
    "2010~2014년",
    "2015~2019년",
    "2020년 이후"
]

df["year_group"] = pd.cut(
    df["건축년도"],
    bins=year_bins,
    labels=year_labels,
    right=True,
    include_lowest=True
)

print("\n[year_group 분포]")
print(df["year_group"].value_counts(dropna=False))

# 비선형 보조 변수
df["year_sq"] = df["건축년도"] ** 2
df["year_sqrt"] = np.sqrt(df["건축년도"])

print("\n[year_sq / year_sqrt 기본 통계]")
print(df[["건축년도", "year_sq", "year_sqrt"]].describe())


[year_group 분포]
year_group
2000~2009년    416429
1990~1999년    368651
1980~1989년    194943
2010~2014년     76992
1980년 이전       41456
2015~2019년     26812
2020년 이후        2811
Name: count, dtype: int64

[year_sq / year_sqrt 기본 통계]
               건축년도       year_sq     year_sqrt
count  1.128094e+06  1.128094e+06  1.128094e+06
mean   1.998791e+03  3.995251e+06  4.470771e+01
std    9.358540e+00  3.738238e+04  1.047053e-01
min    1.961000e+03  3.845521e+06  4.428318e+01
25%    1.992000e+03  3.968064e+06  4.463183e+01
50%    2.000000e+03  4.000000e+06  4.472136e+01
75%    2.005000e+03  4.020025e+06  4.477723e+01
max    2.023000e+03  4.092529e+06  4.497777e+01


- 2-2. 층수 구간 파생변수

In [37]:
# 층수 구간화 함수
def categorize_floor(floor):
    if pd.isna(floor):
        return "unknown"
    try:
        f = int(floor)
    except:
        return "unknown"
    
    if f <= 0:
        return "unknown"
    if f <= 6:
        return "저층(1~6층)"
    elif f <= 15:
        return "중층(7~15층)"
    else:
        return "고층(16층~)"

df["floor_bucket"] = df["층"].apply(categorize_floor)

print("층수 구간 분포:")
print(df["floor_bucket"].value_counts(dropna=False))


층수 구간 분포:
floor_bucket
중층(7~15층)    520740
저층(1~6층)     465851
고층(16층~)     141285
unknown         218
Name: count, dtype: int64


In [38]:
# 층 관련 추가 파생변수
df["floor_abs"] = df["층"].abs()                    # 절댓값 층
df["is_low_floor"] = df["층"].between(1, 6).astype(int)
df["is_mid_floor"] = df["층"].between(7, 15).astype(int)
df["is_high_floor"] = (df["층"] >= 16).astype(int)  # 고층 여부

print("\n[층 관련 추가 파생변수 예시]")
print(df[["층", "floor_abs", "is_low_floor", "is_mid_floor", "is_high_floor"]].head())

print("\n[고층 여부(is_high_floor) 분포]")
print(df["is_high_floor"].value_counts())


[층 관련 추가 파생변수 예시]
   층  floor_abs  is_low_floor  is_mid_floor  is_high_floor
0  3          3             1             0              0
1  4          4             1             0              0
2  5          5             1             0              0
3  4          4             1             0              0
4  2          2             1             0              0

[고층 여부(is_high_floor) 분포]
is_high_floor
0    986809
1    141285
Name: count, dtype: int64


- 2-3. 단지/아파트 단위 파생변수 (최고층, 최상층 여부, 단지 규모)

In [39]:
COMPLEX_ID_COL = "아파트명"  # 예: "아파트명", "단지코드" 등

if COMPLEX_ID_COL in df.columns:
    # 단지별 최고층
    df["max_floor_in_complex"] = df.groupby(COMPLEX_ID_COL)["층"].transform("max")

    # 최상층 여부
    df["is_top_floor"] = (df["층"] == df["max_floor_in_complex"]).astype(int)

    # 단지 규모 구간화 (최고층 기준 예시)
    df["complex_scale"] = pd.cut(
        df["max_floor_in_complex"],
        bins=[0, 10, 20, 30, 70],
        labels=["소형(≤10층)", "중소형(11~20층)", "중대형(21~30층)", "대형(31층~)"],
        include_lowest=True
    )

    print("\n[단지/아파트 단위 파생변수 예시]")
    print(df[[COMPLEX_ID_COL, "층", "max_floor_in_complex", "is_top_floor", "complex_scale"]].head(10))

    print("\n[complex_scale 분포]")
    print(df["complex_scale"].value_counts(dropna=False))
else:
    print(f"\n[경고] 단지 식별 컬럼 {COMPLEX_ID_COL!r} 이(가) df.columns에 없습니다. 실제 컬럼명으로 COMPLEX_ID_COL을 수정하세요.")


[단지/아파트 단위 파생변수 예시]
     아파트명  층  max_floor_in_complex  is_top_floor complex_scale
0  개포6차우성  3                     5             0      소형(≤10층)
1  개포6차우성  4                     5             0      소형(≤10층)
2  개포6차우성  5                     5             1      소형(≤10층)
3  개포6차우성  4                     5             0      소형(≤10층)
4  개포6차우성  2                     5             0      소형(≤10층)
5  개포6차우성  1                     5             0      소형(≤10층)
6  개포6차우성  2                     5             0      소형(≤10층)
7  개포6차우성  5                     5             1      소형(≤10층)
8  개포6차우성  3                     5             0      소형(≤10층)
9  개포6차우성  3                     5             0      소형(≤10층)

[complex_scale 분포]
complex_scale
중소형(11~20층)    593168
중대형(21~30층)    386617
소형(≤10층)        97446
대형(31층~)        50863
Name: count, dtype: int64


- 2-4. 지역(동) 단위 파생변수

In [40]:
DONG_COL = "동"  # 예: "법정동", "동" 등

if DONG_COL in df.columns:
    # 지역별 평균 건물 나이
    df["avg_age_by_dong"] = df.groupby(DONG_COL)["building_age"].transform("mean")

    # 지역별 평균 층수
    df["avg_floor_by_dong"] = df.groupby(DONG_COL)["층"].transform("mean")

    # 지역 내 신축 비율 (예: 2015년 이후 준공 비율)
    df["ratio_new_buildings"] = df.groupby(DONG_COL)["건축년도"].transform(
        lambda x: (x >= 2015).mean()
    )

    print("\n[지역(동) 단위 파생변수 예시]")
    print(df[[DONG_COL, "building_age", "avg_age_by_dong", "avg_floor_by_dong", "ratio_new_buildings"]].head(10))

    print("\n[ratio_new_buildings 상위 20개 지역]")
    print(
        df.groupby(DONG_COL)["ratio_new_buildings"]
          .first()
          .sort_values(ascending=False)
          .head(20)
    )
else:
    print(f"\n[경고] 지역 컬럼 {DONG_COL!r} 이(가) df.columns에 없습니다. 실제 컬럼명으로 DONG_COL을 수정하세요.")



[지역(동) 단위 파생변수 예시]
     동  building_age  avg_age_by_dong  avg_floor_by_dong  ratio_new_buildings
0  개포동            36        36.684946           5.339345              0.03274
1  개포동            36        36.684946           5.339345              0.03274
2  개포동            36        36.684946           5.339345              0.03274
3  개포동            36        36.684946           5.339345              0.03274
4  개포동            36        36.684946           5.339345              0.03274
5  개포동            36        36.684946           5.339345              0.03274
6  개포동            36        36.684946           5.339345              0.03274
7  개포동            36        36.684946           5.339345              0.03274
8  개포동            36        36.684946           5.339345              0.03274
9  개포동            36        36.684946           5.339345              0.03274

[ratio_new_buildings 상위 20개 지역]
동
연지동       1.000000
효제동       1.000000
안암동5가     1.000000
신문로2가     1.000000
충무로5가     1

- 건물 특성 파생변수 전체 요약

In [41]:
# 7. 건물 특성 파생변수 전체 요약
print("\n[건물·층·지역 특성 파생변수 전체 요약]")

cols_summary = [
    "건축년도",
    "building_age",
    "building_age_bucket",
    "year_group",
    "year_sq",
    "year_sqrt",
    "is_new",
    "is_old_redev",
    "층",
    "floor_bucket",
    "floor_abs",
    "is_low_floor",
    "is_mid_floor",
    "is_high_floor",
]

# 단지/지역 관련 컬럼은 존재할 때만 추가
optional_cols = [
    "max_floor_in_complex",
    "is_top_floor",
    "complex_scale",
    "avg_age_by_dong",
    "avg_floor_by_dong",
    "ratio_new_buildings",
]

for c in optional_cols:
    if c in df.columns:
        cols_summary.append(c)

print("\n▶ 기본 통계(describe):")
print(df[cols_summary].describe(include="all"))

print("\n▶ 주요 범주형 변수 분포:")
cat_cols = ["building_age_bucket", "year_group", "floor_bucket", "complex_scale"]
for c in cat_cols:
    if c in df.columns:
        print(f"\n[{c}]")
        print(df[c].value_counts(dropna=False))


[건물·층·지역 특성 파생변수 전체 요약]

▶ 기본 통계(describe):
                건축년도  building_age building_age_bucket  year_group  \
count   1.128094e+06  1.128094e+06             1128094     1128094   
unique           NaN           NaN                   3           7   
top              NaN           NaN         준노후(15~30년)  2000~2009년   
freq             NaN           NaN              660820      416429   
mean    1.998791e+03  2.420950e+01                 NaN         NaN   
std     9.358540e+00  9.358540e+00                 NaN         NaN   
min     1.961000e+03  0.000000e+00                 NaN         NaN   
25%     1.992000e+03  1.800000e+01                 NaN         NaN   
50%     2.000000e+03  2.300000e+01                 NaN         NaN   
75%     2.005000e+03  3.100000e+01                 NaN         NaN   
max     2.023000e+03  6.200000e+01                 NaN         NaN   

             year_sq     year_sqrt        is_new  is_old_redev             층  \
count   1.128094e+06  1.128094e+06

#### Part3. 면적기반

In [42]:
# 전용면적 확인
print("[전용면적 기본 통계]")
print(df["전용면적"].describe())

print("\n전용면적 고유값 상위 20개:")
print(sorted(df["전용면적"].unique())[:20], "...")

[전용면적 기본 통계]
count    1.128094e+06
mean     7.716028e+01
std      2.936448e+01
min      1.002000e+01
25%      5.965000e+01
50%      8.187000e+01
75%      8.496000e+01
max      4.243200e+02
Name: 전용면적, dtype: float64

전용면적 고유값 상위 20개:
[10.02, 10.156, 10.288, 10.3215, 10.78, 11.33, 11.48, 11.6448, 11.6657, 11.79, 11.819, 11.8514, 11.915, 11.9559, 11.9815, 12.0, 12.01, 12.0156, 12.02, 12.03] ...


- 3-1. 전용면적 로그 변환

In [43]:
df["log_area"] = np.log1p(df["전용면적"])   # log1p가 안정적 (log(1+x))

print("\n[log_area 기본 통계]")
print(df["log_area"].describe())


[log_area 기본 통계]
count    1.128094e+06
mean     4.288214e+00
std      3.895357e-01
min      2.399712e+00
25%      4.105120e+00
50%      4.417273e+00
75%      4.453882e+00
max      6.052842e+00
Name: log_area, dtype: float64


- 3-2. 전용면적(㎡) → 평수 변환

In [44]:
df["area_pyeong"] = df["전용면적"] / 3.3

print("\n[평수(area_pyeong) 기본 통계]")
print(df["area_pyeong"].describe())


[평수(area_pyeong) 기본 통계]
count    1.128094e+06
mean     2.338190e+01
std      8.898327e+00
min      3.036364e+00
25%      1.807576e+01
50%      2.480909e+01
75%      2.574545e+01
max      1.285818e+02
Name: area_pyeong, dtype: float64


- 3.3 평수 구간화

In [45]:
def categorize_pyeong(x):
    if pd.isna(x):
        return "unknown"
    if x < 20:
        return "소형(<20평)"
    elif x <= 30:
        return "중형(20~30평)"
    else:
        return "대형(>30평)"

df["area_bucket"] = df["area_pyeong"].apply(categorize_pyeong)

print("\n[area_bucket 분포]")
print(df["area_bucket"].value_counts(dropna=False))


[area_bucket 분포]
area_bucket
중형(20~30평)    476548
소형(<20평)      472710
대형(>30평)      178836
Name: count, dtype: int64


- 3-4. 동 단위 평균 평당가격 파생변수

In [46]:
# train 구간에서만 target 이용 (누수 방지)
df_train = df[df["is_test"] == 0].copy()

df_train["pyp"] = df_train["target"] / (df_train["전용면적"] / 3.3)

dong_pyp = df_train.groupby("동")["pyp"].mean().reset_index()
dong_pyp = dong_pyp.rename(columns={"pyp": "dong_avg_pyp"})

# 전체 df에 merge
df = df.merge(dong_pyp, on="동", how="left")

print(df["dong_avg_pyp"].describe())

count    1.128094e+06
mean     2.455886e+03
std      9.381121e+02
min      7.626487e+02
25%      1.761936e+03
50%      2.113016e+03
75%      2.856487e+03
max      7.645683e+03
Name: dong_avg_pyp, dtype: float64


#### Part4. 거래시점

In [47]:
# 계약년 / 계약월 분리 (YYYYMM 기준)
df["contract_year"] = (df["계약년월"] // 100).astype(int)
df["contract_month"] = (df["계약년월"] % 100).astype(int)

# 분기 (1~4)
df["contract_quarter"] = ((df["contract_month"] - 1) // 3 + 1).astype(int)

# 계절 매핑
def get_season(m):
    if pd.isna(m):
        return "unknown"
    m = int(m)
    if m in [3, 4, 5]:
        return "봄"
    elif m in [6, 7, 8]:
        return "여름"
    elif m in [9, 10, 11]:
        return "가을"
    elif m in [12, 1, 2]:
        return "겨울"
    else:
        return "unknown"

df["season"] = df["contract_month"].apply(get_season)

print("[Time Features 예시]")
print(df[["계약년월", "contract_year", "contract_month", "contract_quarter", "season"]].head())
print(df["season"].value_counts(dropna=False))

[Time Features 예시]
     계약년월  contract_year  contract_month  contract_quarter season
0  201712           2017              12                 4     겨울
1  201712           2017              12                 4     겨울
2  201712           2017              12                 4     겨울
3  201801           2018               1                 1     겨울
4  201801           2018               1                 1     겨울
season
여름    322427
봄     301525
겨울    252232
가을    251910
Name: count, dtype: int64


- 4-2. 구 단위 연간 가격 증가율 (연간 median 기준)

In [48]:
# train 데이터(라벨 있는 부분)만 사용
train_only = df[df["is_test"] == 0].copy()

# 연도 컬럼
train_only["contract_year"] = (train_only["계약년월"] // 100).astype(int)

# 구 + 연도별 median 가격 계산
yearly_median = (
    train_only
    .groupby(["구", "contract_year"])["target"]
    .median()
    .reset_index()
    .rename(columns={"target": "yearly_median_price"})
)

# 같은 구 내에서 연간 증가율 계산 (전년 대비 % 변화)
yearly_median["yearly_price_growth"] = (
    yearly_median
    .sort_values(["구", "contract_year"])
    .groupby("구")["yearly_median_price"]
    .pct_change()   # (올해 - 작년) / 작년
)

# 첫 해는 변화율 없음 → 0으로 채움
yearly_median["yearly_price_growth"] = yearly_median["yearly_price_growth"].fillna(0.0)

print("[구 + 연도별 median 및 증가율 예시]")
print(yearly_median.head())

# 원본 df에 merge (구 + contract_year 키)
df["contract_year"] = (df["계약년월"] // 100).astype(int)
df = df.merge(
    yearly_median[["구", "contract_year", "yearly_price_growth"]],
    on=["구", "contract_year"],
    how="left"
)

# 결측은 0으로 (아주 옛날 연도 등)
df["yearly_price_growth"] = df["yearly_price_growth"].fillna(0.0)

print("\n[yearly_price_growth 기본 통계]")
print(df["yearly_price_growth"].describe())

[구 + 연도별 median 및 증가율 예시]
     구  contract_year  yearly_median_price  yearly_price_growth
0  강남구           2007              56500.0             0.000000
1  강남구           2008              65000.0             0.150442
2  강남구           2009              77800.0             0.196923
3  강남구           2010              75000.0            -0.035990
4  강남구           2011              75000.0             0.000000

[yearly_price_growth 기본 통계]
count    1.128094e+06
mean     9.174233e-02
std      1.049109e-01
min     -4.714567e-01
25%      2.295082e-02
50%      8.031088e-02
75%      1.406250e-01
max      1.566667e+00
Name: yearly_price_growth, dtype: float64


#### Part5. 거래특성

- 5-1. 해제 여부(has_cancelled) 확인

In [49]:
if "has_cancelled" in df.columns:
    print("[has_cancelled 분포]")
    print(df["has_cancelled"].value_counts(dropna=False))
else:
    print("⚠ has_cancelled 컬럼이 없습니다. (01 단계에서 생성 여부 확인 필요)")


[has_cancelled 분포]
has_cancelled
0    1121899
1       6195
Name: count, dtype: int64


- 5-2. 동일 단지 + 동일 면적 반복 거래 횟수 (회전율 개념)

In [50]:
# '단지' 식별을 위해 시/구/동/아파트명 + 전용면적 조합 사용 (필요하면 키 구성 조정 가능)
group_keys = ["시", "구", "동", "아파트명", "전용면적"]

# 전체 데이터 기준으로 같은 단지+면적 조합의 총 거래 수
df["same_apt_area_total_trades"] = (
    df
    .groupby(group_keys)["target"]
    .transform("count")
)

# 계약년월 기준으로 정렬 후, 해당 조합에서 "이전까지 몇 번 거래되었는지"
df = df.sort_values(["계약년월", "계약일"]).reset_index(drop=True)

df["same_apt_area_past_trades"] = (
    df
    .groupby(group_keys)
    .cumcount()   # 0부터 시작 → 과거 거래 횟수
)

print("[거래 특성 파생변수 예시]")
print(df[group_keys + ["same_apt_area_total_trades", "same_apt_area_past_trades"]].head())
print(df["same_apt_area_total_trades"].describe())
print(df["same_apt_area_past_trades"].describe())


[거래 특성 파생변수 예시]
       시    구    동    아파트명   전용면적  same_apt_area_total_trades  \
0  서울특별시  강동구   길동  우림루미아트  84.57                          20   
1  서울특별시  강동구   길동     하이브  59.97                          23   
2  서울특별시  강서구  마곡동      벽산  59.98                         156   
3  서울특별시  노원구  월계동      성원  59.94                         146   
4  서울특별시  노원구  중계동      염광  59.34                         341   

   same_apt_area_past_trades  
0                          0  
1                          0  
2                          0  
3                          0  
4                          0  
count    1.128094e+06
mean     2.431894e+02
std      2.957572e+02
min      0.000000e+00
25%      5.100000e+01
50%      1.420000e+02
75%      3.250000e+02
max      2.148000e+03
Name: same_apt_area_total_trades, dtype: float64
count    1.128094e+06
mean     1.220699e+02
std      1.862697e+02
min      0.000000e+00
25%      1.500000e+01
50%      5.400000e+01
75%      1.490000e+02
max      2.167000e+03
Name: 

#### Part6. 아파트 이름 기반

- 6-1. 브랜드 아파트 여부 (자이/래미안/푸르지오/힐스테이트/e편한세상/롯데캐슬)

In [51]:
brand_keywords = ["자이", "래미안", "푸르지오", "힐스테이트", "e편한세상", "E편한세상", "롯데캐슬"]

def has_brand(name):
    if pd.isna(name):
        return 0
    for kw in brand_keywords:
        if kw in str(name):
            return 1
    return 0

df["is_brand_apt"] = df["아파트명"].apply(has_brand).astype(int)

print("[브랜드 아파트 여부 분포]")
print(df["is_brand_apt"].value_counts(dropna=False))


[브랜드 아파트 여부 분포]
is_brand_apt
0    995036
1    133058
Name: count, dtype: int64


- 6-2. 재건축 가능성 키워드 여부 (주공/한양/삼익/현대1차 등)

In [52]:
redev_keywords = ["주공", "한양", "삼익", "현대1차", "현대 1차"]

def has_redev_keyword(name):
    if pd.isna(name):
        return 0
    for kw in redev_keywords:
        if kw in str(name):
            return 1
    return 0

df["has_redev_keyword"] = df["아파트명"].apply(has_redev_keyword).astype(int)

print("[재건축 키워드 여부 분포]")
print(df["has_redev_keyword"].value_counts(dropna=False))


[재건축 키워드 여부 분포]
has_redev_keyword
0    1026087
1     102007
Name: count, dtype: int64


- 6-3. 단지 규모/프리미엄 이미지 키워드 (센트럴/파크/시티/타워 등)

In [53]:
scale_keywords = ["센트럴", "파크", "시티", "타워", "포레", "하이츠", "팰리스"]

def has_scale_keyword(name):
    if pd.isna(name):
        return 0
    for kw in scale_keywords:
        if kw in str(name):
            return 1
    return 0

df["has_scale_keyword"] = df["아파트명"].apply(has_scale_keyword).astype(int)

print("[단지 규모 키워드 여부 분포]")
print(df["has_scale_keyword"].value_counts(dropna=False))


[단지 규모 키워드 여부 분포]
has_scale_keyword
0    1031171
1      96923
Name: count, dtype: int64


#### Part7. 금리 (추가)

In [54]:
print("▶ 기준금리 raw head")
display(interest.head())

print("\n▶ 주담대 변동금리 raw head")
display(mortgage.head())

▶ 기준금리 raw head


,변환,원자료
0,2007/01,4.5
1,2007/02,4.5
2,2007/03,4.5
3,2007/04,4.5
4,2007/05,4.5



▶ 주담대 변동금리 raw head


,변환,원자료
0,2007/01,6.11
1,2007/02,6.18
2,2007/03,6.20
3,2007/04,6.13
4,2007/05,6.17


In [55]:
# 2) 공통 형식으로 전처리: year, month, 금리값 이름 통일
def preprocess_rate_df(df, rate_col_name):
    """
    df : ['변환', '원자료'] 컬럼을 가진 DataFrame (한국은행 EDS 포맷 가정)
    rate_col_name : 새로 붙일 금리 컬럼명 (예: 'base_rate', 'mortgage_rate')
    """
    out = df.copy()
    out = out.rename(columns={"변환": "ym", "원자료": rate_col_name})
    out["ym"] = out["ym"].astype(str)

    out["year"] = out["ym"].str.slice(0, 4).astype(int)
    out["month"] = out["ym"].str.slice(5, 7).astype(int)

    out = out.drop(columns=["ym"])
    return out[["year", "month", rate_col_name]]

In [56]:
# 3) 기준금리 / 주담대 금리 전처리 (same 변수 이름 계속 사용)
interest = preprocess_rate_df(interest, "base_rate")        # 기준금리
mortgage = preprocess_rate_df(mortgage, "mortgage_rate")    # 주담대

print("\n▶ 기준금리 전처리 결과 (interest)")
display(interest.head())

print("\n▶ 주담대 금리 전처리 결과 (mortgage)")
display(mortgage.head())


▶ 기준금리 전처리 결과 (interest)


,year,month,base_rate
0,2007,1,4.5
1,2007,2,4.5
2,2007,3,4.5
3,2007,4,4.5
4,2007,5,4.5



▶ 주담대 금리 전처리 결과 (mortgage)


,year,month,mortgage_rate
0,2007,1,6.11
1,2007,2,6.18
2,2007,3,6.20
3,2007,4,6.13
4,2007,5,6.17


In [57]:
# 4) 기준금리 + 모기지 금리 merge (year, month 기준)
#    → 최종 금리 테이블도 'interest'라는 이름으로 사용
interest = (
    interest
    .merge(mortgage, on=["year", "month"], how="inner")
    .sort_values(["year", "month"])
    .reset_index(drop=True)
)

print("\n▶ 기준금리 + 주담대 금리 통합 테이블 (interest)")
display(interest.head())



▶ 기준금리 + 주담대 금리 통합 테이블 (interest)


,year,month,base_rate,mortgage_rate
0,2007,1,4.5,6.11
1,2007,2,4.5,6.18
2,2007,3,4.5,6.20
3,2007,4,4.5,6.13
4,2007,5,4.5,6.17


In [58]:
# 5) 금리 파생변수 생성 (interest에 바로 추가)
# 5-1) 기준금리 변화율 (전월 / 전년동월)
interest["base_mom"] = interest["base_rate"].diff(1).fillna(0.0)
interest["base_yoy"] = interest["base_rate"].diff(12).fillna(0.0)

# 5-2) 주담대 금리 변화율 (전월 / 전년동월)
interest["mortgage_mom"] = interest["mortgage_rate"].diff(1).fillna(0.0)
interest["mortgage_yoy"] = interest["mortgage_rate"].diff(12).fillna(0.0)

# 5-3) 스프레드 (모기지 - 기준금리) + 그 전월 변화
interest["spread"] = interest["mortgage_rate"] - interest["base_rate"]
interest["spread_mom"] = interest["spread"].diff(1).fillna(0.0)

# 5-4) 주담대 금리 이동평균(3개월 / 6개월)
interest["mortgage_ma3"] = (
    interest["mortgage_rate"].rolling(window=3, min_periods=1).mean()
)
interest["mortgage_ma6"] = (
    interest["mortgage_rate"].rolling(window=6, min_periods=1).mean()
)

# 5-5) 주담대 금리 시차 변수 (1개월 / 3개월 / 6개월 전 금리)
for lag in [1, 3, 6]:
    col = f"mortgage_lag{lag}"
    interest[col] = interest["mortgage_rate"].shift(lag)

In [59]:
# Lag 초기 NaN → 해당 시점 금리로 채움 (미래 정보 사용 X)
for lag in [1, 3, 6]:
    col = f"mortgage_lag{lag}"
    interest[col] = interest[col].fillna(interest["mortgage_rate"])

In [60]:
# 5-6) 금리 구간 bucket (극저 / 저 / 중 / 고금리)
def categorize_rate(r):
    if r < 1.5:
        return "극저금리"
    elif r < 3.0:
        return "저금리"
    elif r < 4.5:
        return "중금리"
    else:
        return "고금리"

interest["base_rate_bucket"] = interest["base_rate"].apply(categorize_rate)
interest["mortgage_rate_bucket"] = interest["mortgage_rate"].apply(categorize_rate)

print("\n▶ 최종 interest 컬럼 목록")
print(interest.columns.tolist())
display(interest.head())


▶ 최종 interest 컬럼 목록
['year', 'month', 'base_rate', 'mortgage_rate', 'base_mom', 'base_yoy', 'mortgage_mom', 'mortgage_yoy', 'spread', 'spread_mom', 'mortgage_ma3', 'mortgage_ma6', 'mortgage_lag1', 'mortgage_lag3', 'mortgage_lag6', 'base_rate_bucket', 'mortgage_rate_bucket']


,year,month,base_rate,mortgage_rate,base_mom,base_yoy,mortgage_mom,mortgage_yoy,spread,spread_mom,mortgage_ma3,mortgage_ma6,mortgage_lag1,mortgage_lag3,mortgage_lag6,base_rate_bucket,mortgage_rate_bucket
0,2007,1,4.5,6.11,0.0,0.0,0.00,0.0,1.61,0.00,6.110000,6.110000,6.11,6.11,6.11,고금리,고금리
1,2007,2,4.5,6.18,0.0,0.0,0.07,0.0,1.68,0.07,6.145000,6.145000,6.11,6.18,6.18,고금리,고금리
2,2007,3,4.5,6.20,0.0,0.0,0.02,0.0,1.70,0.02,6.163333,6.163333,6.18,6.20,6.20,고금리,고금리
3,2007,4,4.5,6.13,0.0,0.0,-0.07,0.0,1.63,-0.07,6.170000,6.155000,6.20,6.11,6.13,고금리,고금리
4,2007,5,4.5,6.17,0.0,0.0,0.04,0.0,1.67,0.04,6.166667,6.158000,6.13,6.18,6.17,고금리,고금리


In [61]:
# df에 금리 파생변수 merge
#   - 키: (contract_year, contract_month)  ↔  (year, month)

# 1) 계약 연/월 컬럼 생성 (이미 있으면 덮어써도 OK)
df["contract_year"] = (df["계약년월"] // 100).astype(int)
df["contract_month"] = (df["계약년월"] % 100).astype(int)

# 2) merge에 사용할 컬럼 목록
interest_cols_to_merge = [
    "year", "month",
    "base_rate", "base_mom", "base_yoy",
    "mortgage_rate", "mortgage_mom", "mortgage_yoy",
    "spread", "spread_mom",
    "mortgage_ma3", "mortgage_ma6",
    "mortgage_lag1", "mortgage_lag3", "mortgage_lag6",
    "base_rate_bucket", "mortgage_rate_bucket",
]

df = df.merge(
    interest[interest_cols_to_merge],
    left_on=["contract_year", "contract_month"],
    right_on=["year", "month"],
    how="left"
)

# merge용 year, month 정리
df.drop(columns=["year", "month"], inplace=True)

print("\n✅ df에 금리 파생변수 merge 완료")
print("금리 관련 컬럼 예시:")
display(
    df[
        [
            "계약년월", "contract_year", "contract_month",
            "base_rate", "mortgage_rate", "spread",
            "base_mom", "mortgage_mom",
            "mortgage_ma3", "mortgage_lag3",
        ]
    ].head()
)

print("\n현재 df 컬럼 수:", len(df.columns))


✅ df에 금리 파생변수 merge 완료
금리 관련 컬럼 예시:


,계약년월,contract_year,contract_month,base_rate,mortgage_rate,spread,base_mom,mortgage_mom,mortgage_ma3,mortgage_lag3
0,200701,2007,1,4.5,6.11,1.61,0.0,0.0,6.11,6.11
1,200701,2007,1,4.5,6.11,1.61,0.0,0.0,6.11,6.11
2,200701,2007,1,4.5,6.11,1.61,0.0,0.0,6.11,6.11
3,200701,2007,1,4.5,6.11,1.61,0.0,0.0,6.11,6.11
4,200701,2007,1,4.5,6.11,1.61,0.0,0.0,6.11,6.11



현재 df 컬럼 수: 127


#### Log 변환 해보기

In [62]:
# 로그 변환 (스케일 큰 count형 변수 완화)

# 로그 스케일로 줄여줄 대상 컬럼들
log_cols = [
    # 단지 규모 / 세대수
    "k-전체세대수",
    "k-전체동수",
    "주차대수",

    # 구/동 단위 거래량
    "동_거래량",
    "구_거래량",

    # 동일 단지+면적 거래 횟수
    "same_apt_area_total_trades",
    "same_apt_area_past_trades",

    # 구별 학교 수
    "elem_school_cnt",
    "mid_school_cnt",
    "high_school_cnt",

    # 버스 정류장 개수
    "bus_cnt_300m",
    "bus_cnt_500m",

    # 지하철역 개수
    "subway_cnt_500m",
    "subway_cnt_1km",
]

print("\n[로그 변환 대상 컬럼 확인]")
print([c for c in log_cols if c in df.columns])

for col in log_cols:
    if col in df.columns:
        # 음수 보호용 clip(0) 후 log1p 적용
        before_sample = df[col].head(5).values
        df[col] = np.log1p(df[col].clip(lower=0))

        print(f"\n▶ {col} log1p 변환 완료")
        print("   변환 전 예시:", before_sample)
        print("   변환 후 예시:", df[col].head(5).values)


[로그 변환 대상 컬럼 확인]
['k-전체세대수', 'k-전체동수', '주차대수', '동_거래량', '구_거래량', 'same_apt_area_total_trades', 'same_apt_area_past_trades', 'elem_school_cnt', 'mid_school_cnt', 'high_school_cnt', 'bus_cnt_300m', 'bus_cnt_500m', 'subway_cnt_500m', 'subway_cnt_1km']

▶ k-전체세대수 log1p 변환 완료
   변환 전 예시: [ nan  nan  nan 713.  nan]
   변환 후 예시: [       nan        nan        nan 6.57088296        nan]

▶ k-전체동수 log1p 변환 완료
   변환 전 예시: [nan nan nan  6. nan]
   변환 후 예시: [       nan        nan        nan 1.94591015        nan]

▶ 주차대수 log1p 변환 완료
   변환 전 예시: [ nan  nan  nan 390.  nan]
   변환 후 예시: [       nan        nan        nan 5.96870756        nan]

▶ 동_거래량 log1p 변환 완료
   변환 전 예시: [ 8457  8457  2565 17056 26357]
   변환 후 예시: [ 9.04286802  9.04286802  7.85010355  9.74431596 10.17952711]

▶ 구_거래량 log1p 변환 완료
   변환 전 예시: [ 61895  61895  66610 115099 115099]
   변환 후 예시: [11.03321084 11.03321084 11.10662501 11.65355659 11.65355659]

▶ same_apt_area_total_trades log1p 변환 완료
   변환 전 예시: [ 20  23 156 146 341]
   변환 후

#### Data Save

In [63]:
save_path = './Data/02_feature_end'

feature_concat_path = os.path.join(save_path, "feature_engineered_concat.pkl")

with open(feature_concat_path, "wb") as f:
    pickle.dump(df, f)

print("저장 경로 : ", feature_concat_path)
print("Shape : ", df.shape)

저장 경로 :  ./Data/02_feature_end/feature_engineered_concat.pkl
Shape :  (1128094, 127)


In [64]:
# 분리저장
# is_test 기준으로 다시 분리
train_df = df[df["is_test"] == 0].copy()
test_df  = df[df["is_test"] == 1].copy()

# 분리된 'train_df', 'test_df'의 shape를 확인
print("train_df shape:", train_df.shape)
print("test_df  shape:", test_df.shape)

# LGBM용으로 바로 쓸 pkl 저장 (03_model에서 불러올 용도)
train_fe_path = "./Data/02_feature_end/train_fe.pkl"
test_fe_path  = "./Data/02_feature_end/test_fe.pkl"

# 여기서도 반드시 'train_df', 'test_df' 저장!
train_df.to_pickle(train_fe_path)
test_df.to_pickle(test_fe_path)

print("train 저장 완료 : ", train_fe_path)
print("test  저장 완료 : ", test_fe_path)


train_df shape: (1118822, 127)
test_df  shape: (9272, 127)
train 저장 완료 :  ./Data/02_feature_end/train_fe.pkl
test  저장 완료 :  ./Data/02_feature_end/test_fe.pkl
